In [1]:
class chamber:
    '''Class for chambers'''
    def __init__(self, name, serial, cal_factor_dict):
        self.name = name
        self.serial = serial                   # could be a string or a number
        self.cal_factor = cal_factor_dict      # Nk, function of HVL but pass as dict
    
    def calibration(self, kV):                 # ignore fact that cal is in HVL
        if kV in self.cal_factor.keys():
            return self.cal_factor[kV]
        else:
            print('No cal factor for that kV')
        

In [2]:
Farmer = chamber('farmer', 3627, {80 : 4.143, 140 :4.143, 250 :4.079}) # init

In [3]:
Farmer.calibration(140)

4.143

In [4]:
class measurement:
    '''Generic class of measurement. Readings must be a list.
    Daily correction is the correction to apply to readings. E.g. if daily output is 2% low, this factor is 1.02
    Temperature in degrees Celcius, Pressure in mmHg'''
    
    def __init__(self, chamber, readings, daily_correction, temperature, pressure, kV):
        self.chamber= chamber       # of class chamber
        self.readings = readings    # should be a list
        self.daily_correction = daily_correction
        self.temperature = temperature
        self.pressure = pressure
        self.kV = kV
        self.Nk = self.chamber.calibration(self.kV)  # get the cal factor at the kV
        self.ftp = ((self.temperature + 273.15)*760.0)/(293.15* self.pressure)
    
    def Add_Reading(self, reading):
        self.readings.append(reading)
    
    def Mean_Reading(self):
        return sum(self.readings)/len(self.readings)    # should be a list

    def Corrected_Mean_Reading(self):
        return self.Mean_Reading()*self.daily_correction*self.ftp
     

In [5]:
output1 = measurement(Farmer, [1.0, 2.0, 3.0], 1.02, 25.0, 745.0, 80)

In [6]:
output1.Add_Reading(5.0)

In [7]:
output1.Corrected_Mean_Reading()

2.9102821813969304

In [8]:
class medium_kV_output(measurement):                 # subclass measurement and add backscatter
    '''Sub class measurement with kV, requires backscatter'''
    
    def __init__(self, chamber, readings, daily_correction, temperature, pressure, kV, backscatter_factor):
        super().__init__(chamber, readings, daily_correction, temperature, pressure, kV)   # init the superclass
        self.backscatter_factor = backscatter_factor
        
    def Get_Output(self):
        return self.Corrected_Mean_Reading()*self.backscatter_factor*self.Nk  # should add mass-energy              

In [9]:
kV_output1 = medium_kV_output(Farmer, [1.0, 2.0, 3.0], 1.02, 25.0, 745.0, 80, 1.068)

In [10]:
kV_output1.Get_Output()

9.365233028944983